In [16]:
import ipywidgets as widgets
from ipyleaflet import Map, WidgetControl
import geopandas as gpd

In [26]:
# Path to your shapefile
shapefile_path = './county_boundary/cb_2018_us_county_5m.shp'

# Reading the shapefile
gdf = gpd.read_file(shapefile_path)
projected_gdf = gdf.to_crs(epsg=2163)



In [14]:
state_fips_mapping = {
    '01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA',
    '08': 'CO', '09': 'CT', '10': 'DE', '12': 'FL', '13': 'GA',
    '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA',
    '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD',
    '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO',
    '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH', '34': 'NJ',
    '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH',
    '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC',
    '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT',
    '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI', '56': 'WY'
}

In [35]:
# Update state names in the GeoDataFrame using the mapping
gdf['StateName'] = gdf['STATEFP'].apply(lambda x: state_fips_mapping.get(x, ''))

# Dropdown for states
state_dropdown = widgets.Dropdown(
    options=[''] + sorted(gdf['StateName'].unique()),
    description='State:',
    disabled=False,
)

# Dropdown for counties (initially empty)
county_dropdown = widgets.Dropdown(
    description='County:',
    disabled=True,  # Disabled until a state is selected
)

def update_county_dropdown(*args):
    selected_state = state_dropdown.value
    if selected_state:
        filtered_gdf = gdf[gdf['StateName'] == selected_state]
        county_dropdown.options = [''] + sorted(filtered_gdf['NAME'].unique())
        county_dropdown.disabled = False
    else:
        county_dropdown.options = []
        county_dropdown.disabled = True

state_dropdown.observe(update_county_dropdown, 'value')

# Initialize the map
mymap = Map(center=(37.0902, -95.7129), zoom=3)  # Default center of the US

def on_county_change(change):
    selected_county = change['new']
    if selected_county:
        # Filter the projected GeoDataFrame for the selected county
        county_gdf_projected = projected_gdf[projected_gdf['NAME'] == selected_county]

        # Calculate the centroid in the projected CRS
        centroid_projected = county_gdf_projected.geometry.centroid.iloc[0]

        # Since centroid_projected is in the projected CRS, we need to transform its coordinates back to EPSG:4326
        # Create a GeoSeries with the centroid and transform it
        centroid_latlon = gpd.GeoSeries([centroid_projected], crs=projected_gdf.crs).to_crs(epsg=4326).iloc[0]

        lat, lon = centroid_latlon.y, centroid_latlon.x

        # Update the map view without adding a new marker
        mymap.center = (lat, lon)
        mymap.zoom = 12

county_dropdown.observe(on_county_change, names='value')

# Tool for the toolbox
tool = widgets.VBox([state_dropdown, county_dropdown])

# Create a WidgetControl for the toolbox tool
toolbox_control = WidgetControl(widget=tool, position='topright')

# Add the toolbox control to the map
mymap.add_control(toolbox_control)

In [36]:
mymap


Map(center=[37.0902, -95.7129], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…